<h2>Cross validation </h2>

the idea is to use not only a training and test set, but also a validation set. Using a test set to adapt the Model is kind of cheating, that means that the model will more or less overfit the test set whereas it should not.
For each iteration we can also split the validation set (k-fold cross validation set - or even better Nested K-fold validation set) .If 5 iterations, validation set is 1/5th of the total training set.
Validation Set allow to evaluate performance of the model.


In [11]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier

iris = load_iris()
X, y = iris.data, iris.target

classifier = KNeighborsClassifier()

# Warning, the dataset is sorted, so, plitting it would result in a bad range of values, especially targets
import numpy as np
rng = np.random.RandomState(0)

permutation = rng.permutation(len(X))

X, y = X[permutation], y[permutation]

k = 5
n_samples = len(X)
fold_size = n_samples // k
scores = []
masks = []

for fold in range(k):
    # generate a boolean mask for the test set in this fold
    test_mask = np.zeros(n_samples, dtype=bool)
    test_mask[fold * fold_size : (fold + 1) * fold_size] = True

    # store the mask for visualization
    masks.append(test_mask)
    
    # create training and test set using this mask
    X_test, y_test = X[test_mask], y[test_mask]
    X_train, y_train = X[~test_mask], y[~test_mask]
    
    # fit the classifier
    classifier.fit(X_train, y_train)
    
    # Compute the score and record it
    scores.append(classifier.score(X_test, y_test))
    